In [1]:
#Import Dependencies
import pandas as pd
import numpy as np
from babel.numbers import format_currency

In [2]:
#Set up variables for filepaths
filepathschools = "../Resources/schools_complete.csv"
filepathstudents = "../Resources/students_complete.csv"

In [3]:
# Read CSV Files into Pandas
schools_df = pd.read_csv(filepathschools, encoding="UTF-8")
students_df = pd.read_csv(filepathstudents, encoding="UTF-8")

In [5]:
# Show header for school data
schools_df.head()

,School ID,school_name,type,size,budget
0,0,Huang High School,District,2917,1910635
1,1,Figueroa High School,District,2949,1884411
2,2,Shelton High School,Charter,1761,1056600
3,3,Hernandez High School,District,4635,3022020
4,4,Griffin High School,Charter,1468,917500


In [6]:
# Show header for student data
students_df.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score
0,0,Paul Bradley,M,9th,Huang High School,66,79
1,1,Victor Smith,M,12th,Huang High School,94,61
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58
4,4,Bonnie Ray,F,9th,Huang High School,97,84


In [7]:
#District Summary Information
######### Calculate each value to be used in dataframe, then build out the dataframe below  #####

#Total School Count
#schools_df['School ID'].count()

#Total Student Count
#students_df['Student ID'].count()

#Total Budget Amount
#schools_df['budget'].sum()

#Student Average Math Score
#students_df['math_score'].mean()

#Student Average Reading Score
#students_df['reading_score'].mean()


#Percent of Students Passing Math Score
StudentsPassMath_df=students_df.loc[students_df['math_score']>=70,:]
#StudentsPassMath_df['Student ID'].count()
#((StudentsPassMath_df['Student ID'].count())/(students_df['Student ID'].count()))*100.00


#Percent of Students Passing Reading Score
StudentsPassReading_df=students_df.loc[students_df['reading_score']>=70,:]
#StudentsPassReading_df['Student ID'].count()
#((StudentsPassReading_df['Student ID'].count())/(students_df['Student ID'].count()))*100.00


#Percent of Students Passing Math AND Reading Score
StudentsPassMathANDReading_df = students_df.where((students_df['math_score'] >= 70) & (students_df['reading_score'] >= 70))
StudentsPassMathANDReading_df=StudentsPassMathANDReading_df.dropna(how='any')
#StudentsPassMathANDReading_df.head()
#StudentsPassMathANDReading_df['Student ID'].count()
#((StudentsPassMathANDReading_df['Student ID'].count())/(students_df['Student ID'].count()))*100.00


#Build Data Frame District Summary Information
DistrictSummary_df = pd.DataFrame({
    "School_Count": [schools_df['School ID'].count()],
    "Student_Count": [students_df['Student ID'].count()],
    "Budget_Amount": [schools_df['budget'].sum()],
    "Avg_Math_Score": [students_df['math_score'].mean()],
    "Avg_Reading_Score": [students_df['reading_score'].mean()],
    "Pct_Pass_Math": [((StudentsPassMath_df['Student ID'].count())/(students_df['Student ID'].count()))*100.00],
    "Pct_Pass_Reading": [((StudentsPassReading_df['Student ID'].count())/(students_df['Student ID'].count()))*100.00],
    "Pct_Pass_Math_AND_Reading": [((StudentsPassMathANDReading_df['Student ID'].count())/(students_df['Student ID'].count()))*100.00]})


#Rename Overall Column
DistrictSummary_df = DistrictSummary_df.rename(columns={'Pct_Pass_Math_AND_Reading': 'Pct_Overall_Passing'})


#Format Total Budget Amount Column
DistrictSummary_df["Budget_Amount"] = DistrictSummary_df["Budget_Amount"].apply(lambda x: format_currency(x, currency="USD", locale="en_US"))

#Final Data Review
DistrictSummary_df


,School_Count,Student_Count,Budget_Amount,Avg_Math_Score,Avg_Reading_Score,Pct_Pass_Math,Pct_Pass_Reading,Pct_Overall_Passing
0,15,39170,"$24,649,428.00",78.985371,81.87784,74.980853,85.805463,65.172326


In [8]:
#School Summary Information

#Build Data Frame School Summary Information
SchoolSummary_df = schools_df
   
#Rename Columns 
SchoolSummary_df = SchoolSummary_df.rename(columns={'school_name': 'School_Name', 'type': 'School_Type', 'size': 'Total_Students', 'budget': 'School_Budget'})
 
#New Calculated Column for per student budget 
SchoolSummary_df['Per_Student_Budget']=(SchoolSummary_df['School_Budget']/SchoolSummary_df['Total_Students'])

#Format Total Budget Amount Column
SchoolSummary_df["School_Budget"] = SchoolSummary_df["School_Budget"].apply(lambda x: format_currency(x, currency="USD", locale="en_US"))
SchoolSummary_df["Per_Student_Budget"] = SchoolSummary_df["Per_Student_Budget"].apply(lambda x: format_currency(x, currency="USD", locale="en_US"))

#Re-define datatype needed for later merge operation
SchoolSummary_df['School_Name'] = SchoolSummary_df['School_Name'].astype(str)

#Final Data Review
#SchoolSummary_df

#Math Scores Grouped By School
ListofMathScoresBySchool_df = students_df[['school_name','math_score']]
AverageMathScoresBySchool_df = ListofMathScoresBySchool_df.groupby(['school_name'], as_index=True).mean()
AverageMathScoresBySchool_df.reset_index(inplace=True)
AverageMathScoresBySchool_df['school_name'] = AverageMathScoresBySchool_df['school_name'].astype(str)
#AverageMathScoresBySchool_df.head()
#print(AverageMathScoresBySchool_df)    
 
#Reading Scores Grouped By School
ListofReadingScoresBySchool_df = students_df[['school_name','reading_score']]
AverageReadingScoresBySchool_df = ListofReadingScoresBySchool_df.groupby(['school_name'], as_index=True).mean()
AverageReadingScoresBySchool_df.reset_index(inplace=True)
AverageReadingScoresBySchool_df['school_name'] = AverageReadingScoresBySchool_df['school_name'].astype(str)
#AverageReadingScoresBySchool_df.head()
#print(AverageReadingScoresBySchool_df)     
    
    
#Merge School Summary Dataframe with Averages Dataframes
SumWMath_df=pd.merge(SchoolSummary_df, AverageMathScoresBySchool_df,how="inner", left_on='School_Name', right_on='school_name')
del SumWMath_df['school_name']
#SumWMath_df.head()
SumWRead_df=pd.merge(SumWMath_df, AverageReadingScoresBySchool_df,how="inner", left_on='School_Name', right_on='school_name')
del SumWRead_df['school_name']
#SumWRead_df.head()

#Rename Columns
SumWRead_df = SumWRead_df.rename(columns={'math_score': 'Avg_Math_Score', 'reading_score': 'Avg_Reading_Score'})
#SumWRead_df.head()


#Count of Students Passing Math Score
StudentsPassMath_df=students_df.loc[students_df['math_score']>=70,['school_name','math_score']]
#StudentsPassMath_df.head()
CountPassMathScoresBySchool_df = StudentsPassMath_df.groupby(['school_name'], as_index=True).count()
CountPassMathScoresBySchool_df.reset_index(inplace=True)
CountPassMathScoresBySchool_df['school_name'] = CountPassMathScoresBySchool_df['school_name'].astype(str)
CountPassMathScoresBySchool_df = CountPassMathScoresBySchool_df.rename(columns={'math_score': 'Count_Passed_Math'})
#CountPassMathScoresBySchool_df.head()


#Count of Students Passing Reading Score
StudentsPassReading_df=students_df.loc[students_df['reading_score']>=70,['school_name','reading_score']]
#StudentsPassReading_df.head()
CountPassReadingScoresBySchool_df = StudentsPassReading_df.groupby(['school_name'], as_index=True).count()
CountPassReadingScoresBySchool_df.reset_index(inplace=True)
CountPassReadingScoresBySchool_df['school_name'] = CountPassReadingScoresBySchool_df['school_name'].astype(str)
CountPassReadingScoresBySchool_df = CountPassReadingScoresBySchool_df.rename(columns={'reading_score': 'Count_Passed_Reading'})
#CountPassReadingScoresBySchool_df.head()


#Count of Students Passing Math AND Reading Score
StudentsPassMathANDReading_df=students_df.where((students_df['math_score'] >= 70) & (students_df['reading_score'] >= 70))
StudentsPassMathANDReading_df=StudentsPassMathANDReading_df.dropna(how='any')
#StudentsPassMathANDReading_df.head()
CountPassMathANDReadingScoresBySchool_df = StudentsPassMathANDReading_df.groupby(['school_name'], as_index=True).count()
CountPassMathANDReadingScoresBySchool_df.reset_index(inplace=True)
CountPassMathANDReadingScoresBySchool_df['school_name'] = CountPassMathANDReadingScoresBySchool_df['school_name'].astype(str)
CountPassMathANDReadingScoresBySchool_df = CountPassMathANDReadingScoresBySchool_df.rename(columns={'Student ID': 'Count_Passed_Math_AND_Reading'})
del CountPassMathANDReadingScoresBySchool_df['student_name']
del CountPassMathANDReadingScoresBySchool_df['gender']
del CountPassMathANDReadingScoresBySchool_df['grade']
del CountPassMathANDReadingScoresBySchool_df['reading_score']
del CountPassMathANDReadingScoresBySchool_df['math_score']
#CountPassMathANDReadingScoresBySchool_df.head()


#"Pct_Pass_Math": ['N/A'],
#"Pct_Pass_Reading": ['N/A'],
#"Pct_Pass_Math_AND_Reading": ['N/A']})

#Merge Tables and calculate Percentage for Math
SumWMathCount_df=pd.merge(SumWRead_df, CountPassMathScoresBySchool_df,how="inner", left_on='School_Name', right_on='school_name')
del SumWMathCount_df['school_name']
SumWMathCount_df['Pct_Passed_Math']=((SumWMathCount_df['Count_Passed_Math'])/(SumWMathCount_df['Total_Students']))*100.00
#SumWMathCount_df.head()

#Merge Tables and calculate Percentage for Reading
SumWReadingCount_df=pd.merge(SumWMathCount_df, CountPassReadingScoresBySchool_df,how="inner", left_on='School_Name', right_on='school_name')
del SumWReadingCount_df['school_name']
SumWReadingCount_df['Pct_Passed_Reading']=((SumWReadingCount_df['Count_Passed_Reading'])/(SumWReadingCount_df['Total_Students']))*100.00
#SumWReadingCount_df.head()


#Merge Tables and calculate Percentage for MathANDReading
SumWMathANDReadingCount_df=pd.merge(SumWReadingCount_df, CountPassMathANDReadingScoresBySchool_df,how="inner", left_on='School_Name', right_on='school_name')
del SumWMathANDReadingCount_df['school_name']
SumWMathANDReadingCount_df['Pct_Passed_Math_AND_Reading']=((SumWMathANDReadingCount_df['Count_Passed_Math_AND_Reading'])/(SumWMathANDReadingCount_df['Total_Students']))*100.00
#SumWMathANDReadingCount_df.head()

# Remove Count Columns
del SumWMathANDReadingCount_df['Count_Passed_Math']
del SumWMathANDReadingCount_df['Count_Passed_Reading']
del SumWMathANDReadingCount_df['Count_Passed_Math_AND_Reading']

#Rename Overall Column
SumWMathANDReadingCount_df = SumWMathANDReadingCount_df.rename(columns={'Pct_Passed_Math_AND_Reading': 'Pct_Overall_Passing'})

#Clean Up Label
FinalDataFrame = SumWMathANDReadingCount_df


#Final Data Pull 
FinalDataFrame.head(15)

,School ID,School_Name,School_Type,Total_Students,School_Budget,Per_Student_Budget,Avg_Math_Score,Avg_Reading_Score,Pct_Passed_Math,Pct_Passed_Reading,Pct_Overall_Passing
0,0,Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
1,1,Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
2,2,Shelton High School,Charter,1761,"$1,056,600.00",$600.00,83.359455,83.725724,93.867121,95.854628,89.892107
3,3,Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
4,4,Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
5,5,Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,90.582567
6,6,Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
7,7,Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,54.642283
8,8,Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.505855,96.252927,89.227166
9,9,Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541


In [9]:
#Top 5 Performing Schools
TopPerformingSchools_df = FinalDataFrame.sort_values(['Pct_Overall_Passing'], ascending=False)
TopPerformingSchools_df.head(5)

,School ID,School_Name,School_Type,Total_Students,School_Budget,Per_Student_Budget,Avg_Math_Score,Avg_Reading_Score,Pct_Passed_Math,Pct_Passed_Reading,Pct_Overall_Passing
6,6,Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
14,14,Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.418349,83.848930,93.272171,97.308869,90.948012
4,4,Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
5,5,Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,90.582567
9,9,Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541


In [10]:
#Bottom Performing Schools
BottomPerformingSchools_df = FinalDataFrame.sort_values(['Pct_Overall_Passing'], ascending=True)
BottomPerformingSchools_df.head(5)

,School ID,School_Name,School_Type,Total_Students,School_Budget,Per_Student_Budget,Avg_Math_Score,Avg_Reading_Score,Pct_Passed_Math,Pct_Passed_Reading,Pct_Overall_Passing
11,11,Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,52.988247
1,1,Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
0,0,Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
3,3,Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
12,12,Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172


In [11]:
#Math Scores By Grade
MathScoresByGrade_df = students_df[['school_name','grade','math_score']]
#MathScoresByGrade_df.head()
MathScoreAverageByGrade_df = MathScoresByGrade_df.groupby(['school_name','grade']).mean()
#Rename Math Score Column
MathScoreAverageByGrade_df = MathScoreAverageByGrade_df.rename(columns={'math_score': 'Average_Math_Score'})
MathScoreAverageByGrade_df.reset_index(inplace=True)
#MathScoreAverageByGrade_df.head(60)

#9th Grade DataFrame
MathScore9th_df=MathScoreAverageByGrade_df.loc[MathScoreAverageByGrade_df['grade'] == '9th',['school_name','grade','Average_Math_Score']]
MathScore9th_df = MathScore9th_df.rename(columns={'Average_Math_Score': '9th'})
del MathScore9th_df['grade']
#MathScore9th_df.head(15)

#10th Grade DataFrame
MathScore10th_df=MathScoreAverageByGrade_df.loc[MathScoreAverageByGrade_df['grade'] == '10th',['school_name','grade','Average_Math_Score']]
MathScore10th_df = MathScore10th_df.rename(columns={'Average_Math_Score': '10th'})
del MathScore10th_df['grade']
#MathScore10th_df.head(15)

#11th Grade DataFrame
MathScore11th_df=MathScoreAverageByGrade_df.loc[MathScoreAverageByGrade_df['grade'] == '11th',['school_name','grade','Average_Math_Score']]
MathScore11th_df = MathScore11th_df.rename(columns={'Average_Math_Score': '11th'})
del MathScore11th_df['grade']
#MathScore11th_df.head(15)

#12th Grade DataFrame
MathScore12th_df=MathScoreAverageByGrade_df.loc[MathScoreAverageByGrade_df['grade'] == '12th',['school_name','grade','Average_Math_Score']]
MathScore12th_df = MathScore12th_df.rename(columns={'Average_Math_Score': '12th'})
del MathScore12th_df['grade']
#MathScore12th_df.head(15)

#Merge DataFrames
Sum10MathAvg_df=pd.merge(MathScore9th_df, MathScore10th_df,how="inner", left_on='school_name', right_on='school_name')
#del Sum10MathAvg_df['school_name']
#Sum10MathAvg_df.head(15)

Sum11MathAvg_df=pd.merge(Sum10MathAvg_df, MathScore11th_df,how="inner", left_on='school_name', right_on='school_name')
#del Sum11MathAvg_df['school_name']
Sum11MathAvg_df.head(15)

FinalByGradeMathAvg_df=pd.merge(Sum11MathAvg_df, MathScore12th_df,how="inner", left_on='school_name', right_on='school_name')
#del FinalByGradeMathAvg_df['school_name']
FinalByGradeMathAvg_df.head(15)



,school_name,9th,10th,11th,12th
0,Bailey High School,77.083676,76.996772,77.515588,76.492218
1,Cabrera High School,83.094697,83.154506,82.765560,83.277487
2,Figueroa High School,76.403037,76.539974,76.884344,77.151369
3,Ford High School,77.361345,77.672316,76.918058,76.179963
4,Griffin High School,82.044010,84.229064,83.842105,83.356164
5,Hernandez High School,77.438495,77.337408,77.136029,77.186567
6,Holden High School,83.787402,83.429825,85.000000,82.855422
7,Huang High School,77.027251,75.908735,76.446602,77.225641
8,Johnson High School,77.187857,76.691117,77.491653,76.863248
9,Pena High School,83.625455,83.372000,84.328125,84.121547


In [12]:
#Reading Scores By Grade
ReadingScoresByGrade_df = students_df[['school_name','grade','reading_score']]
#ReadingScoresByGrade_df.head()

ReadingScoreAverageByGrade_df = ReadingScoresByGrade_df.groupby(['school_name','grade']).mean()
#Rename Reading Score Column
ReadingScoreAverageByGrade_df = ReadingScoreAverageByGrade_df.rename(columns={'reading_score': 'Average_Reading_Score'})
ReadingScoreAverageByGrade_df.reset_index(inplace=True)
#ReadingScoreAverageByGrade_df.head(60)


#9th Grade DataFrame
ReadingScore9th_df=ReadingScoreAverageByGrade_df.loc[ReadingScoreAverageByGrade_df['grade'] == '9th',['school_name','grade','Average_Reading_Score']]
ReadingScore9th_df = ReadingScore9th_df.rename(columns={'Average_Reading_Score': '9th'})
del ReadingScore9th_df['grade']
#ReadingScore9th_df.head(15)

#10th Grade DataFrame
ReadingScore10th_df=ReadingScoreAverageByGrade_df.loc[ReadingScoreAverageByGrade_df['grade'] == '10th',['school_name','grade','Average_Reading_Score']]
ReadingScore10th_df = ReadingScore10th_df.rename(columns={'Average_Reading_Score': '10th'})
del ReadingScore10th_df['grade']
#ReadingScore10th_df.head(15)

#11th Grade DataFrame
ReadingScore11th_df=ReadingScoreAverageByGrade_df.loc[ReadingScoreAverageByGrade_df['grade'] == '11th',['school_name','grade','Average_Reading_Score']]
ReadingScore11th_df = ReadingScore11th_df.rename(columns={'Average_Reading_Score': '11th'})
del ReadingScore11th_df['grade']
#ReadingScore11th_df.head(15)

#12th Grade DataFrame
ReadingScore12th_df=ReadingScoreAverageByGrade_df.loc[ReadingScoreAverageByGrade_df['grade'] == '12th',['school_name','grade','Average_Reading_Score']]
ReadingScore12th_df = ReadingScore12th_df.rename(columns={'Average_Reading_Score': '12th'})
del ReadingScore12th_df['grade']
#ReadingScore12th_df.head(15)

#Merge DataFrames
Sum10ReadingAvg_df=pd.merge(ReadingScore9th_df, ReadingScore10th_df,how="inner", left_on='school_name', right_on='school_name')
#del Sum10MathAvg_df['school_name']
#Sum10ReadingAvg_df.head(15)

Sum11ReadingAvg_df=pd.merge(Sum10ReadingAvg_df,ReadingScore11th_df,how="inner", left_on='school_name', right_on='school_name')
#del Sum11ReadingAvg_df['school_name']
#Sum11ReadingAvg_df.head(15)

FinalByGradeReadingAvg_df=pd.merge(Sum11ReadingAvg_df, ReadingScore12th_df,how="inner", left_on='school_name', right_on='school_name')
#del FinalByGradeReadingAvg_df['school_name']
FinalByGradeReadingAvg_df.head(15)




,school_name,9th,10th,11th,12th
0,Bailey High School,81.303155,80.907183,80.945643,80.912451
1,Cabrera High School,83.676136,84.253219,83.788382,84.287958
2,Figueroa High School,81.198598,81.408912,80.640339,81.384863
3,Ford High School,80.632653,81.262712,80.403642,80.662338
4,Griffin High School,83.369193,83.706897,84.288089,84.013699
5,Hernandez High School,80.866860,80.660147,81.396140,80.857143
6,Holden High School,83.677165,83.324561,83.815534,84.698795
7,Huang High School,81.290284,81.512386,81.417476,80.305983
8,Johnson High School,81.260714,80.773431,80.616027,81.227564
9,Pena High School,83.807273,83.612000,84.335938,84.591160


In [13]:
#Scores By School Spending

#Build Data Frame School Summary Information
SchoolSpendSummary_df = schools_df
   
#Rename Columns 
SchoolSpendSummary_df = SchoolSpendSummary_df.rename(columns={'school_name': 'School_Name', 'type': 'School_Type', 'size': 'Total_Students', 'budget': 'School_Budget'})
 
#New Calculated Column for per student budget 
SchoolSpendSummary_df['Per_Student_Budget']=(SchoolSpendSummary_df['School_Budget']/SchoolSpendSummary_df['Total_Students'])

#Re-define datatype needed for later merge operation
SchoolSpendSummary_df['School_Name'] = SchoolSpendSummary_df['School_Name'].astype(str)

#Sort the dataframe
SchoolSpendSummary_df = SchoolSpendSummary_df.sort_values(['Per_Student_Budget'], ascending=False)

#Final Data Review
#SchoolSpendSummary_df

#Bin the schools by Per Student Budget Value
bins = [ 575, 600, 625, 650 , 675 ]
labels = ['575 - 600', '601 - 625', '626 - 650','651 - 675']
SchoolSpendSummary_df['Budget_Group'] = pd.cut(SchoolSpendSummary_df['Per_Student_Budget'] , bins=bins, labels=labels, include_lowest=True)

#Remove extraneous columns
del SchoolSpendSummary_df['School ID']
del SchoolSpendSummary_df['School_Type']
del SchoolSpendSummary_df['Total_Students']
del SchoolSpendSummary_df['School_Budget']
del SchoolSpendSummary_df['Per_Student_Budget']


#Final Data Review
#SchoolSpendSummary_df.head(15)

#Merge School Spend Summary Dataframe with Student Dataframes
SchoolStudentBudgetGroup_df=pd.merge(SchoolSpendSummary_df, students_df,how="inner", left_on='School_Name', right_on='school_name')
SchoolStudentBudgetGroup_df.reset_index(inplace=True)
#SchoolStudentBudgetGroup_df.head()


 
#Bin the Reading Score
binsRead = [ 0, 69, 100 ]
labelsRead = [0, 1]
SchoolStudentBudgetGroup_df['Read_Status'] = pd.cut(SchoolStudentBudgetGroup_df['reading_score'] , bins=binsRead, labels=labelsRead, include_lowest=True)
#SchoolStudentBudgetGroup_df.head()

#Bin the Math Score
binsMath = [ 0, 69, 100 ]
labelsMath = [0, 1]
SchoolStudentBudgetGroup_df['Math_Status'] = pd.cut(SchoolStudentBudgetGroup_df['math_score'] , bins=binsMath, labels=labelsMath, include_lowest=True)
#SchoolStudentBudgetGroup_df.head()

#Re-define datatype needed for later addition operation
SchoolStudentBudgetGroup_df['Read_Status'] = SchoolStudentBudgetGroup_df['Read_Status'].astype(int)
SchoolStudentBudgetGroup_df['Math_Status'] = SchoolStudentBudgetGroup_df['Math_Status'].astype(int)

#Set Overall Passing Status Value
SchoolStudentBudgetGroup_df['Overall_Status'] = np.where((SchoolStudentBudgetGroup_df['Read_Status']+SchoolStudentBudgetGroup_df['Math_Status']==2), 1, 0)

#Final Data Review
SchoolStudentBudgetGroup_df.head()

#Remove Extraneous Columns
del SchoolStudentBudgetGroup_df['School_Name']
del SchoolStudentBudgetGroup_df['student_name']
del SchoolStudentBudgetGroup_df['gender']
del SchoolStudentBudgetGroup_df['grade']
del SchoolStudentBudgetGroup_df['school_name']

#Checkscript
#SchoolStudentBudgetGroup_df.head(30000)

#Aggregate Counts for Percentage Calculations
BudgetGroupPrecentages_df=SchoolStudentBudgetGroup_df.groupby(
   ['Budget_Group']
).agg(
    {
         'Student ID': 'count',           
         'Read_Status': sum,    
         'Math_Status': sum,  
         'Overall_Status': sum
    }
)

#Rename Total Students Count
BudgetGroupPrecentages_df = BudgetGroupPrecentages_df.rename(columns={'Student ID': 'Total_Students'})

   
#New Calculated Columns for Percentages 
BudgetGroupPrecentages_df['Pct_Pass_Math']=(BudgetGroupPrecentages_df['Math_Status']/BudgetGroupPrecentages_df['Total_Students'])*100.00
BudgetGroupPrecentages_df['Pct_Pass_Reading']=(BudgetGroupPrecentages_df['Read_Status']/BudgetGroupPrecentages_df['Total_Students'])*100.00
BudgetGroupPrecentages_df['Pct_Pass_Overall']=(BudgetGroupPrecentages_df['Overall_Status']/BudgetGroupPrecentages_df['Total_Students'])*100.00

#Remove Extraneous Columns
del BudgetGroupPrecentages_df['Total_Students']
del BudgetGroupPrecentages_df['Read_Status']
del BudgetGroupPrecentages_df['Math_Status']
del BudgetGroupPrecentages_df['Overall_Status']

#Budget Group Percentage Information
#BudgetGroupPrecentages_df.head()

#Raw Data for Aggregating Average/Mode
#SchoolStudentBudgetGroup_df.head()

#New Dataframe
BudgetGroupList_df=SchoolStudentBudgetGroup_df

del BudgetGroupList_df['index']
del BudgetGroupList_df['Student ID']
del BudgetGroupList_df['Read_Status']
del BudgetGroupList_df['Math_Status']
del BudgetGroupList_df['Overall_Status']

#BudgetGroupList_df.head()

#Aggregate Counts for Percentage Calculations
BudgetGroupAverages_df=BudgetGroupList_df.groupby(
   ['Budget_Group']
).agg(
    {
         ('Average',np.mean), 
    }
)

#Drop second level descriptor of averages
BudgetGroupAverages_df=BudgetGroupAverages_df.droplevel(1, axis=1)

#Rename Total Students Count
BudgetGroupAverages_df = BudgetGroupAverages_df.rename(columns={'reading_score': 'Reading_Score_Avg'})
BudgetGroupAverages_df = BudgetGroupAverages_df.rename(columns={'math_score': 'Math_Score_Avg'})

#Checkscript
#BudgetGroupAverages_df.head()
#BudgetGroupPrecentages_df.head()

#Final Dataframe Merge
FinalBudgetGroupStats_df=pd.merge(BudgetGroupAverages_df, BudgetGroupPrecentages_df,how="inner", left_on='Budget_Group', right_on='Budget_Group')
FinalBudgetGroupStats_df.head(15)



,Reading_Score_Avg,Math_Score_Avg,Pct_Pass_Math,Pct_Pass_Reading,Pct_Pass_Overall
Budget_Group,,,,,
575 - 600,83.912412,83.362283,93.738467,96.506335,90.478534
601 - 625,83.906996,83.544856,93.868313,96.666667,90.576132
626 - 650,81.162258,77.469253,68.659481,82.131155,56.650363
651 - 675,81.030323,77.034693,66.340042,81.038136,53.522246


In [14]:
#Scores By School Size

#Build Data Frame School Summary Information
SchoolSizeSummary_df = schools_df
   
#Rename Columns 
SchoolSizeSummary_df = SchoolSizeSummary_df.rename(columns={'school_name': 'School_Name', 'type': 'School_Type', 'size': 'Total_Students', 'budget': 'School_Budget'})
 
#New Calculated Column for per student budget 
SchoolSizeSummary_df['Per_Student_Budget']=(SchoolSizeSummary_df['School_Budget']/SchoolSizeSummary_df['Total_Students'])

#Re-define datatype needed for later merge operation
SchoolSizeSummary_df['School_Name'] = SchoolSizeSummary_df['School_Name'].astype(str)

#Sort the dataframe
SchoolSizeSummary_df = SchoolSizeSummary_df.sort_values(['Total_Students'], ascending=False)

#Final Data Review
#SchoolSizeSummary_df

#Bin the schools by Total Students
bins = [ 0, 1800, 3600, 5000 ]
labels = ['0 - 1800', '1801 - 3600', '3601 - 5000']
SchoolSizeSummary_df['Size_Group'] = pd.cut(SchoolSizeSummary_df['Total_Students'] , bins=bins, labels=labels, include_lowest=True)


#Remove extraneous columns
del SchoolSizeSummary_df['School ID']
del SchoolSizeSummary_df['School_Type']
del SchoolSizeSummary_df['Total_Students']
del SchoolSizeSummary_df['School_Budget']
del SchoolSizeSummary_df['Per_Student_Budget']


#Final Data Review
#SchoolSizeSummary_df.head(15)


#Merge School Size Summary Dataframe with Student Dataframes
SchoolStudentSizeGroup_df=pd.merge(SchoolSizeSummary_df, students_df,how="inner", left_on='School_Name', right_on='school_name')
SchoolStudentSizeGroup_df.reset_index(inplace=True)
#SchoolStudentSizeGroup_df.head()

 
#Bin the Reading Score
binsRead = [ 0, 69, 100 ]
labelsRead = [0, 1]
SchoolStudentSizeGroup_df['Read_Status'] = pd.cut(SchoolStudentSizeGroup_df['reading_score'] , bins=binsRead, labels=labelsRead, include_lowest=True)
#SchoolStudentSizeGroup_df.head()

#Bin the Math Score
binsMath = [ 0, 69, 100 ]
labelsMath = [0, 1]
SchoolStudentSizeGroup_df['Math_Status'] = pd.cut(SchoolStudentSizeGroup_df['math_score'] , bins=binsMath, labels=labelsMath, include_lowest=True)
#SchoolStudentSizeGroup_df.head()

#Re-define datatype needed for later addition operation
SchoolStudentSizeGroup_df['Read_Status'] = SchoolStudentSizeGroup_df['Read_Status'].astype(int)
SchoolStudentSizeGroup_df['Math_Status'] = SchoolStudentSizeGroup_df['Math_Status'].astype(int)

#Set Overall Passing Status Value
SchoolStudentSizeGroup_df['Overall_Status'] = np.where((SchoolStudentSizeGroup_df['Read_Status']+SchoolStudentSizeGroup_df['Math_Status']==2), 1, 0)

#Final Data Review
SchoolStudentSizeGroup_df.head()

#Remove Extraneous Columns
del SchoolStudentSizeGroup_df['School_Name']
del SchoolStudentSizeGroup_df['student_name']
del SchoolStudentSizeGroup_df['gender']
del SchoolStudentSizeGroup_df['grade']
del SchoolStudentSizeGroup_df['school_name']

#Checkscript
#SchoolStudentSizeGroup_df.head(30000)


#Aggregate Counts for Percentage Calculations
SizeGroupPrecentages_df=SchoolStudentSizeGroup_df.groupby(
   ['Size_Group']
).agg(
    {
         'Student ID': 'count',           
         'Read_Status': sum,    
         'Math_Status': sum,  
         'Overall_Status': sum
    }
)

#Rename Total Students Count
SizeGroupPrecentages_df = SizeGroupPrecentages_df.rename(columns={'Student ID': 'Total_Students'})

#Budget Group Percentage Information
#SizeGroupPrecentages_df.head()
   
#New Calculated Columns for Percentages 
SizeGroupPrecentages_df['Pct_Pass_Math']=(SizeGroupPrecentages_df['Math_Status']/SizeGroupPrecentages_df['Total_Students'])*100.00
SizeGroupPrecentages_df['Pct_Pass_Reading']=(SizeGroupPrecentages_df['Read_Status']/SizeGroupPrecentages_df['Total_Students'])*100.00
SizeGroupPrecentages_df['Pct_Pass_Overall']=(SizeGroupPrecentages_df['Overall_Status']/SizeGroupPrecentages_df['Total_Students'])*100.00

#Remove Extraneous Columns
del SizeGroupPrecentages_df['Total_Students']
del SizeGroupPrecentages_df['Read_Status']
del SizeGroupPrecentages_df['Math_Status']
del SizeGroupPrecentages_df['Overall_Status']

#Budget Group Percentage Information
#SizeGroupPrecentages_df.head()

#Raw Data for Aggregating Average/Mode
#SchoolStudentSizeGroup_df.head()

#New Dataframe
SizeGroupList_df=SchoolStudentSizeGroup_df

del SizeGroupList_df['index']
del SizeGroupList_df['Student ID']
del SizeGroupList_df['Read_Status']
del SizeGroupList_df['Math_Status']
del SizeGroupList_df['Overall_Status']

SizeGroupList_df.head()

#Aggregate Counts for Percentage Calculations
SizeGroupAverages_df=SizeGroupList_df.groupby(
   ['Size_Group']
).agg(
    {
         ('Average',np.mean), 
    }
)

#Drop second level descriptor of averages
SizeGroupAverages_df=SizeGroupAverages_df.droplevel(1, axis=1)


#Rename Total Students Count
SizeGroupAverages_df = SizeGroupAverages_df.rename(columns={'reading_score': 'Reading_Score_Avg'})
SizeGroupAverages_df = SizeGroupAverages_df.rename(columns={'math_score': 'Math_Score_Avg'})

#Checkscript
#SizeGroupAverages_df.head()
#SizeGroupPrecentages_df.head()

#Final Dataframe Merge
FinalSizeGroupStats_df=pd.merge(SizeGroupAverages_df, SizeGroupPrecentages_df,how="inner", left_on='Size_Group', right_on='Size_Group')
FinalSizeGroupStats_df.head(15)


,Reading_Score_Avg,Math_Score_Avg,Pct_Pass_Math,Pct_Pass_Reading,Pct_Pass_Overall
Size_Group,,,,,
0 - 1800,83.861418,83.523035,93.555197,96.585124,90.376257
1801 - 3600,81.993096,78.878001,75.513887,85.768084,65.761808
3601 - 5000,80.928365,77.070764,66.468891,81.106091,53.715094


In [15]:
#Scores By School Type

#Build Data Frame School Summary Information
SchoolTypeSummary_df = schools_df
   
#Rename Columns 
SchoolTypeSummary_df = SchoolTypeSummary_df.rename(columns={'school_name': 'School_Name', 'type': 'School_Type', 'size': 'Total_Students', 'budget': 'School_Budget'})
 
#New Calculated Column for per student budget 
SchoolTypeSummary_df['Per_Student_Budget']=(SchoolTypeSummary_df['School_Budget']/SchoolTypeSummary_df['Total_Students'])

#Re-define datatype needed for later merge operation
SchoolTypeSummary_df['School_Name'] = SchoolTypeSummary_df['School_Name'].astype(str)

#Sort the dataframe
SchoolTypeSummary_df = SchoolTypeSummary_df.sort_values(['School_Type'], ascending=False)

#Final Data Review
SchoolTypeSummary_df

#Remove extraneous columns
del SchoolTypeSummary_df['School ID']
del SchoolTypeSummary_df['Total_Students']
del SchoolTypeSummary_df['School_Budget']
del SchoolTypeSummary_df['Per_Student_Budget']

#Final Data Review
#SchoolTypeSummary_df.head(15)


#Merge School Size Summary Dataframe with Student Dataframes
SchoolStudentTypeGroup_df=pd.merge(SchoolTypeSummary_df, students_df,how="inner", left_on='School_Name', right_on='school_name')
SchoolStudentTypeGroup_df.reset_index(inplace=True)
SchoolStudentTypeGroup_df.head()

 
#Bin the Reading Score
binsRead = [ 0, 69, 100 ]
labelsRead = [0, 1]
SchoolStudentTypeGroup_df['Read_Status'] = pd.cut(SchoolStudentTypeGroup_df['reading_score'] , bins=binsRead, labels=labelsRead, include_lowest=True)
#SchoolStudentTypeGroup_df.head()

#Bin the Math Score
binsMath = [ 0, 69, 100 ]
labelsMath = [0, 1]
SchoolStudentTypeGroup_df['Math_Status'] = pd.cut(SchoolStudentTypeGroup_df['math_score'] , bins=binsMath, labels=labelsMath, include_lowest=True)
SchoolStudentTypeGroup_df.head()

#Re-define datatype needed for later addition operation
SchoolStudentTypeGroup_df['Read_Status'] = SchoolStudentTypeGroup_df['Read_Status'].astype(int)
SchoolStudentTypeGroup_df['Math_Status'] = SchoolStudentTypeGroup_df['Math_Status'].astype(int)

#Set Overall Passing Status Value
SchoolStudentTypeGroup_df['Overall_Status'] = np.where((SchoolStudentTypeGroup_df['Read_Status']+SchoolStudentTypeGroup_df['Math_Status']==2), 1, 0)

#Final Data Review
#SchoolStudentTypeGroup_df.head()

#Remove Extraneous Columns
del SchoolStudentTypeGroup_df['index']
del SchoolStudentTypeGroup_df['School_Name']
del SchoolStudentTypeGroup_df['student_name']
del SchoolStudentTypeGroup_df['gender']
del SchoolStudentTypeGroup_df['grade']
del SchoolStudentTypeGroup_df['school_name']

#Checkscript
#SchoolStudentTypeGroup_df.head(30000)




#Aggregate Counts for Percentage Calculations
TypePrecentages_df=SchoolStudentTypeGroup_df.groupby(
   ['School_Type']
).agg(
    {
         'Student ID': 'count',           
         'Read_Status': sum,    
         'Math_Status': sum,  
         'Overall_Status': sum
    }
)

#Rename Total Students Count
TypePrecentages_df = TypePrecentages_df.rename(columns={'Student ID': 'Total_Students'})

#Budget Group Percentage Information
#TypePrecentages_df.head()
   
#New Calculated Columns for Percentages 
TypePrecentages_df['Pct_Pass_Math']=(TypePrecentages_df['Math_Status']/TypePrecentages_df['Total_Students'])*100.00
TypePrecentages_df['Pct_Pass_Reading']=(TypePrecentages_df['Read_Status']/TypePrecentages_df['Total_Students'])*100.00
TypePrecentages_df['Pct_Pass_Overall']=(TypePrecentages_df['Overall_Status']/TypePrecentages_df['Total_Students'])*100.00

#Remove Extraneous Columns
del TypePrecentages_df['Total_Students']
del TypePrecentages_df['Read_Status']
del TypePrecentages_df['Math_Status']
del TypePrecentages_df['Overall_Status']

#Budget Group Percentage Information
#TypePrecentages_df.head()

#Raw Data for Aggregating Average/Mode
#SchoolStudentTypeGroup_df.head()

#New Dataframe
TypeGroupList_df=SchoolStudentTypeGroup_df

del TypeGroupList_df['Student ID']
del TypeGroupList_df['Read_Status']
del TypeGroupList_df['Math_Status']
del TypeGroupList_df['Overall_Status']

TypeGroupList_df.head()

#Aggregate Counts for Percentage Calculations
TypeGroupAverages_df=TypeGroupList_df.groupby(
   ['School_Type']
).agg(
    {
         ('Average',np.mean), 
    }
)

#Drop second level descriptor of averages
TypeGroupAverages_df=TypeGroupAverages_df.droplevel(1, axis=1)


#Rename Total Students Count
TypeGroupAverages_df = TypeGroupAverages_df.rename(columns={'reading_score': 'Reading_Score_Avg'})
TypeGroupAverages_df = TypeGroupAverages_df.rename(columns={'math_score': 'Math_Score_Avg'})

#Checkscript
#TypeGroupAverages_df.head()
TypePrecentages_df.head()

#Final Dataframe Merge
FinalTypeStats_df=pd.merge(TypeGroupAverages_df, TypePrecentages_df,how="inner", left_on='School_Type', right_on='School_Type')
FinalTypeStats_df.head()


,Reading_Score_Avg,Math_Score_Avg,Pct_Pass_Math,Pct_Pass_Reading,Pct_Pass_Overall
School_Type,,,,,
Charter,83.902821,83.406183,93.701821,96.645891,90.560932
District,80.962485,76.987026,66.518387,80.905249,53.695878
